In [1]:
# Dependencies
from urllib.parse import urlencode, urlparse, parse_qsl
import numpy as np
import pandas as pd
import gmaps
import requests
import matplotlib.pyplot as plt
from census import Census
from us import states

pd.set_option('display.max_colwidth', 30)
pd.options.display.float_format = '{:,.2f}'.format

# Census & gmaps API Keys
from config import (census_key, g_key)

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# Read in the csv containing state centroid coordinates
centroids = pd.read_csv("resources/state_centroids.csv")

In [3]:
# Run Census Search to retrieve data on all states
# Note the addition of "B23025_005E" for unemployment count
year_list = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]
census_sum_pd = pd.DataFrame()
for year in range(len(year_list)):
    year_request = year_list[year]
    c = Census(census_key, year=year_request)
    census_data =c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E"), {'for': 'state:*'})
    census_pd = pd.DataFrame(census_data)
    # Column Reordering
    census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Med_Age",
                                      "B19013_001E": "House_Income",
                                      "B19301_001E": "Per_Cap_Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "State_Name", "state": "State"})

    # Add in Poverty Rate (Poverty Count / Population)
    census_pd["Pov_Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

    # Add in Employment Rate (Employment Count / Population)
    census_pd["Unemp_Rate"] = 100 * \
    census_pd["Unemployment Count"].astype(
        int) / census_pd["Population"].astype(int)
    census_pd['Year'] = year_request
    print('Retrieval data for:', year_request)
    census_sum_pd = pd.concat([census_sum_pd, census_pd], axis=0)

Retrieval data for: 2011
Retrieval data for: 2012
Retrieval data for: 2013
Retrieval data for: 2014
Retrieval data for: 2015
Retrieval data for: 2016
Retrieval data for: 2017
Retrieval data for: 2018


In [4]:
# Merge the datasets using the state columns
census_data = pd.merge(census_sum_pd, centroids, how="left", left_on="State_Name", right_on="State")
census_data.drop(['Poverty Count', 'Unemployment Count', 'State_x', 'State_y'], axis=1, inplace=True)
census_data = census_data.rename(columns={'Latitude': 'Lat', 'Longitude':'Lng'})

In [5]:
# Excluding Outlier
census_data.drop(census_data[census_data['State_Name']=='Puerto Rico'].index, inplace=True)
northen_states_list = ['Alaska', 'Connecticut', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Maine', 'Massachusetts',
                       'Michigan', 'Minnesota', 'Missouri', 'Nebraska', 'New Hampshire', 'New Jersey',
                       'New York', 'North Dakota', 'Ohio', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Dakota',
                       'Vermont', 'Wisconsin', 'Wyoming']
census_data['Geo']='South'
for row in census_data.itertuples():
    if row.State_Name in northen_states_list:
        census_data.loc[row.Index, 'Geo'] = 'North'
census_data.head()

,State_Name,House_Income,Population,Med_Age,Per_Cap_Income,Pov_Rate,Unemp_Rate,Year,Lat,Lng,Geo
0,Alabama,"42,934.00","4,747,424.00",37.70,"23,483.00",17.13,4.54,2011,32.78,-86.83,South
1,Alaska,"69,014.00","700,703.00",33.80,"31,944.00",9.29,4.41,2011,64.07,-152.28,North
2,Arizona,"50,752.00","6,337,373.00",35.70,"25,784.00",15.84,4.23,2011,34.27,-111.66,South
3,Arkansas,"40,149.00","2,895,928.00",37.30,"21,833.00",17.85,3.94,2011,34.89,-92.44,South
4,California,"61,632.00","36,969,200.00",35.10,"29,634.00",14.10,5.06,2011,37.18,-119.47,South


In [6]:
census_data.groupby(['State_Name', 'Year']).sum()

House_Income   Population  Med_Age  Per_Cap_Income  Pov_Rate  \
State_Name Year                                                                 
Alabama    2011     42,934.00 4,747,424.00    37.70       23,483.00     17.13   
           2012     43,160.00 4,777,326.00    37.80       23,587.00     17.63   
           2013     43,253.00 4,799,277.00    38.10       23,680.00     18.14   
           2014     43,511.00 4,817,678.00    38.20       23,936.00     18.47   
           2015     43,623.00 4,830,620.00    38.40       24,091.00     18.37   
...                       ...          ...      ...             ...       ...   
Wyoming    2014     58,252.00   575,251.00    36.80       29,381.00     11.35   
           2015     58,840.00   579,679.00    36.80       29,803.00     11.21   
           2016     59,143.00   583,029.00    36.80       30,139.00     11.28   
           2017     60,938.00   583,200.00    37.00       31,214.00     10.87   
           2018     62,268.00   581,836.00    37.30       32,295.00     10.88   

                 Unemp_Rate   Lat     Lng  
State_Name Year                            
Alabama    2011        4.54 32.78  -86.83  
           2012        4.83 32.78  -86.83  
           2013        5.04 32.78  -86.83  
           2014        4.75 32.78  -86.83  
           2015        4.29 32.78  -86.83  
...                     ...   ...     ...  
Wyoming    2014        2.77 43.00 -107.55  
           2015        2.59 43.00 -107.55  
           2016        2.59 43.00 -107.55  
           2017        2.48 43.00 -107.55  
           2018        2.35 43.00 -107.55  

[408 rows x 8 columns]

In [7]:
# Save as a csv
census_data_2011 = census_data[census_data['Year']==2011]
census_data_2011.to_csv("resources/states_2011.csv", encoding="utf-8", index=False)
census_data.to_csv("resources/states_total.csv", encoding="utf-8", index=False)

In [8]:
# Exploring Census DataSet form dsource zbp
year='2010'
dsource='zbp'
cols='ESTAB,EMP,PAYANN'
state='17'
base_url = f'https://api.census.gov/data/{year}/{dsource}'
data_url = f'{base_url}?get={cols}&for=zipcode:*&ST={state}&key={census_key}'
response=requests.get(data_url)
illinois = pd.DataFrame(response.json())
illinois.columns = illinois.iloc[0]
illinois.drop(0, axis=0, inplace=True)
illinois.sort_values(by='zipcode')

,ESTAB,EMP,PAYANN,ST,zipcode
1,682,3022,83703,17,11419
2,55,416,11094,17,17853
3,12,0,0,17,60001
4,586,4429,142143,17,60002
5,1906,19854,1067994,17,60010
...,...,...,...,...,...
1483,7,140,4694,17,62998
1484,16,165,3102,17,62999
1485,1047,17392,698924,17,76180
1486,105,1503,37804,17,76182
